In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # wrapper to use function from sklearn
from tensorflow.keras import backend as be
from tensorflow.keras import layers, Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV #training and testing data split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import multiprocessing as mp

# import plotting libraries
import matplotlib.pyplot as plt
#import sns for better plots, it is handy to manage subplots
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [51]:
import os
for dirname, _, filenames in os.walk('C:\Projects\Python\MNIST\data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"



C:\Projects\Python\MNIST\data\test.csv
C:\Projects\Python\MNIST\data\train.csv


In [52]:
data = pd.read_csv("C:/Projects/Python/MNIST/data/train.csv")
data.head(5)

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#       tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.10)
#   except RuntimeError as e:
#     print(e)
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session());

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# tf.config.list_physical_devices('GPU') 
# print(tf.config.list_physical_devices('GPU') )
# print('tf is build with cuda: ' + str(tf.test.is_built_with_cuda()))

In [62]:
train, test= train_test_split(data, test_size=0.15, stratify=data["label"])

Xtrain = train.drop('label', axis=1).values
Xtrain_norm = Xtrain/255.
Xtrain_final = np.reshape(Xtrain_norm, (-1,28,28,1))

ohe = OneHotEncoder()
ytrain = train['label'].values
ytrain_trans = ohe.fit_transform(np.reshape(ytrain, (-1,1)))
print(ytrain)
ytrain = to_categorical(ytrain, 10)
print(Xtrain_final.shape)
print(ytrain.shape)

[6 4 7 ... 4 9 1]
(35700, 28, 28, 1)
(35700, 10)


In [56]:
def create_model(optimizer, learning_rate, first_layer_filters, second_layer_filters, first_kernel, second_kernel):
    if optimizer=='SGD':
        optimizer = SGD(learning_rate = learning_rate)
    if optimizer=='RMSprop':
        optimizer = RMSprop(learning_rate = learning_rate)
    if optimizer=='Adagrad':
        optimizer = Adagrad(learning_rate = learning_rate)
    if optimizer=='Adadelta':
        optimizer = Adadelta(learning_rate = learning_rate)
    if optimizer=='Adam':
        optimizer = Adam(learning_rate = learning_rate)
    if optimizer=='Adamax':
        optimizer = Adamax(learning_rate = learning_rate)
    if optimizer=='Nadam':
        optimizer = Nadam(learning_rate = learning_rate)
    
    model = Sequential()
    model.add(tf.keras.Input(shape=(28,28,1)))
    model.add(layers.Conv2D(15, first_kernel, strides=(2, 2), activation = 'relu'))
    model.add(layers.Conv2D(second_layer_filters, second_kernel,strides=(2, 2), activation = 'relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    
    #model.add(layers.Dropout(0.2))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model



In [48]:
sk_model = KerasClassifier(build_fn=create_model, verbose=0)

k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

optimizer = ['Nadam'] # SGD', 'RMSprop', 'Adagrad', 'Adadelta',
learning_rate = [0.01]
batch_size = [75]
epochs = [3]
first_layer_filters = [50]
second_layer_filters = [25]
first_kernel = [(3,3)]
second_kernel = [(5,5)]
parameter_grid = dict(learning_rate=learning_rate, optimizer=optimizer, batch_size=batch_size, epochs=epochs, first_layer_filters=first_layer_filters, second_layer_filters=second_layer_filters, first_kernel=first_kernel, second_kernel=second_kernel)

gs = GridSearchCV(sk_model, parameter_grid, cv=k_fold, scoring='accuracy', n_jobs=10, verbose=4)
grid_result = gs.fit(Xtrain_final, ytrain)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Fitting 5 folds for each of 16 candidates, totalling 80 fits
# Best: 0.972521 using {'batch_size': 30, 'epochs': 3, 'first_kernel': (3, 3), 'first_layer_filters': 20, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (3, 3), 'second_layer_filters': 10}

#Fitting 5 folds for each of 64 candidates, totalling 320 fits
#Best: 0.974594 using {'batch_size': 30, 'epochs': 3, 'first_kernel': (4, 4), 'first_layer_filters': 20, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (4, 4), 'second_layer_filters': 20}

#first_layer_filters = [30, 40]
#second_layer_filters = [20, 30]
#first_kernel = [(4,4), (5,5)]
#second_kernel = [(4,4), (5,5)]
#Fitting 5 folds for each of 32 candidates, totalling 160 fits
#Best: 0.974538 using {'batch_size': 30, 'epochs': 3, 'first_kernel': (4, 4), 'first_layer_filters': 40, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 20}

#optimizer = ['Nadam'] # SGD', 'RMSprop', 'Adagrad', 'Adadelta',
#learning_rate = [0.01]
#batch_size = [30, 40]
#epochs = [3]
#first_layer_filters = [40, 50]
#second_layer_filters = [15, 20, 25]
#first_kernel = [(4,4)]
#second_kernel = [(5,5), (6,6)]
#Fitting 5 folds for each of 24 candidates, totalling 120 fits
#Best: 0.975378 using {'batch_size': 40, 'epochs': 3, 'first_kernel': (4, 4), 'first_layer_filters': 40, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 25}

#batch_size = [40, 50, 60]
#epochs = [3]
#first_layer_filters = [40, 45]
#second_layer_filters = [25, 30]
#Fitting 5 folds for each of 12 candidates, totalling 60 fits
#Best: 0.978319 using {'batch_size': 60, 'epochs': 3, 'first_kernel': (4, 4), 'first_layer_filters': 45, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 25}

#learning_rate = [0.01, 0.03]
#batch_size = [60, 70, 80, 90]
#epochs = [3]
#first_layer_filters = [45, 55]
#second_layer_filters = [20, 25]
#Fitting 5 folds for each of 32 candidates, totalling 160 fits
#Best: 0.979692 using {'batch_size': 70, 'epochs': 3, 'first_kernel': (4, 4), 'first_layer_filters': 45, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 25}

#learning_rate = [0.01, 0.02]
#batch_size = [65, 70, 75]
#epochs = [3]
#first_layer_filters = [45, 50]
#second_layer_filters = [25, 27]
#first_kernel = [(3,3), (4,4)]
#second_kernel = [(4,4),(5,5)]
#Fitting 5 folds for each of 96 candidates, totalling 480 fits
#Best: 0.979916 using {'batch_size': 75, 'epochs': 3, 'first_kernel': (3, 3), 'first_layer_filters': 50, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 25}

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best: 0.979916 using {'batch_size': 75, 'epochs': 3, 'first_kernel': (3, 3), 'first_layer_filters': 50, 'learning_rate': 0.01, 'optimizer': 'Nadam', 'second_kernel': (5, 5), 'second_layer_filters': 25}


In [63]:
start_time = time.time()
opti_model = create_model('Nadam', 0.01, 50, 25, (3,3), (5,5))
opti_model.fit(Xtrain_final, ytrain, batch_size=75, epochs=3, shuffle=True)
print('end time: ' + str(time.time()-start_time))

(35700, 28, 28, 1)
(35700, 10)
Epoch 1/3
476/476 [==============================] - 2s 3ms/step - loss: 0.3694 - accuracy: 0.8844
Epoch 2/3
476/476 [==============================] - 1s 3ms/step - loss: 0.0702 - accuracy: 0.9793
Epoch 3/3
476/476 [==============================] - 1s 3ms/step - loss: 0.0493 - accuracy: 0.9849
end time: 4.4982805252075195


In [84]:
test_data = pd.read_csv("C:/Projects/Python/MNIST/data/test.csv")
print(test_data)

       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
27995       0       0       0       0       0       0       0       0       0   
27996       0       0       0       0       0       0       0       0       0   
27997       0       0       0       0       0       0       0       0       0   
27998       0       0       0       0       0       0       0       0       0   
27999       0       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel77

In [85]:
Xtrain = test_data.values
Xtrain_norm = Xtrain/255.
Xtrain_final = np.reshape(Xtrain_norm, (-1,28,28,1))

print(Xtrain_final.shape)

(28000, 28, 28, 1)


In [87]:
ypred = opti_model.predict(Xtrain_final)
print(ypred)
print(ypred.shape)
ypred = np.argmax(ypred, axis=1)
print(ypred)
#print(result)
result = pd.DataFrame()
result['ImageId'] = range(1,28001)
result['Label'] = ypred
print(result.head())
result.to_csv(r'C:/Projects/Python/MNIST/data/my_submission.csv', index=False, header=True)

[[9.9531472e-13 3.4410890e-12 1.0000000e+00 ... 2.9823016e-11
  2.3583682e-10 7.9463481e-14]
 [9.9999976e-01 3.1381104e-12 1.1592147e-08 ... 1.3794098e-10
  1.3425328e-10 6.3648237e-10]
 [1.1149791e-10 2.6325782e-08 2.1310120e-08 ... 6.5227852e-07
  5.7361089e-04 9.9932480e-01]
 ...
 [9.7601182e-16 7.7563413e-09 6.9638445e-10 ... 4.4974588e-09
  6.5497954e-09 4.4775696e-08]
 [4.0482625e-08 2.6637701e-09 1.2442715e-09 ... 1.3633336e-04
  7.0167818e-07 9.9915278e-01]
 [6.9427381e-10 2.2692139e-09 9.9866867e-01 ... 1.0176349e-09
  1.3302482e-03 8.0080326e-08]]
(28000, 10)
[2 0 9 ... 3 9 2]
   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3
